In [2]:
#导入第三方库
import torch 
from torch import nn
from torch.nn import functional as F

In [3]:
#通过实例化nn.Sequential来构建模型
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0145,  0.1447,  0.1822,  0.0292, -0.0920,  0.1992,  0.2526, -0.2102,
         -0.1267, -0.4142],
        [-0.0752,  0.2277,  0.0867,  0.0730, -0.2226,  0.1559,  0.2617, -0.1474,
         -0.1195, -0.2591]], grad_fn=<AddmmBackward0>)

自定义块
总结每个块必须提供的基本功能：
- 将输入数据作为其前向传播函数的参数
- 通过前向传播函数生成输出(输出形状可能与输入形状不一样)
- 计算其输出关于输入的梯度，可通过其反向传播函数进行访问
- 存储和访问前向传播计算所需的参数
- 根据需要初始化模型参数

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) #定义隐藏层
        self.out = nn.Linear(256, 10) #定义输出层
    
    def forward(self, X):
        #使用ReLU的函数版本，在nn.functional中有定义
        return self.out(F.relu(self.hidden(X)))

In [5]:
#系统自动生成反向传播函数和模型初始参数
net = MLP()
net(X)

tensor([[-3.3070e-01,  5.0582e-02, -5.9760e-02,  3.4124e-01, -1.7196e-01,
          9.1782e-02,  4.6077e-02,  1.0822e-02, -1.4946e-01,  4.9948e-02],
        [-2.0232e-01,  4.1026e-02, -5.7754e-02,  3.0041e-01, -1.7775e-01,
          5.0201e-02, -1.7272e-04,  5.0000e-02, -1.2342e-01, -1.1324e-02]],
       grad_fn=<AddmmBackward0>)

顺序快：为了实现Sequential类的工作原理，我们需要定义两个关键函数
- 一种将块逐个追加到列表中的函数
- 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的链条

In [6]:
#实现MySequential类
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            #这里, module是Module子类的一个实例，我们把它保存在'Module'类的成员
            #变量_modules中，类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        #OrderedDict保证了按照成员顺序遍历
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1692,  0.0806,  0.0305, -0.1654, -0.0244,  0.0998,  0.0786, -0.0175,
          0.3414, -0.3511],
        [-0.0794, -0.0040,  0.0482, -0.0190, -0.0482,  0.1406,  0.1012,  0.0808,
          0.1870, -0.2224]], grad_fn=<AddmmBackward0>)

In [8]:
#在前向传播函数中执行代码，实现FixedHiddenMLP类
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        #不计算梯度的随机权重参数，不进行参数更新
        self.rand_weight = torch.rand((20, 20), requires_grad= False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        #使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X , self.rand_weight) + 1)
        X = self.linear(X)
        #控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1896, grad_fn=<SumBackward0>)

In [11]:
#混合搭配各种组合块的用法
class NestMLP(nn.Module):
    def __init__(self):
        #继承父类的初始化方法
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64),
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1247, grad_fn=<SumBackward0>)